In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
from sklearn.preprocessing import OneHotEncoder
import time
from arcface import calculate_arcface_logits

<br>

Load Data

In [2]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train),(x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)
ohe = OneHotEncoder()
y_train_ohe = ohe.fit_transform(y_train.reshape(-1,1)).toarray().astype('float32')
y_test_ohe = ohe.transform(y_test.reshape(-1,1)).toarray().astype('float32')
print('x_train:{}, y_train:{}'.format(x_train.shape, y_train.shape))
print('x_test:{}, y_test:{}'.format(x_test.shape, y_test.shape))
print('y_train_ohe:', y_train_ohe.shape)
print('y_test_ohe:', y_test_ohe.shape)

x_train:(50000, 32, 32, 3), y_train:(50000,)
x_test:(10000, 32, 32, 3), y_test:(10000,)
y_train_ohe: (50000, 10)
y_test_ohe: (10000, 10)


/home/kim1/anaconda3/envs/rok/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


<br>

params

In [3]:
params = {}
params['seed'] = 0
params['embedding'] = 16
params['n_classes'] = 10
params['labels'] = np.unique(y_train).astype('int')
params['batch_size'] = 128
params['logits_scale'] = 10
params['logits_margin'] = 0.1
params['feed_limit'] = 10000

<br>
Model = LeNet5

In [4]:
import tensorflow.contrib.slim as slim
tf.reset_default_graph()
tf.set_random_seed(params['seed'])

x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y_ohe = tf.placeholder(tf.float32, [None, params['n_classes']])

conv1 = tf.layers.conv2d(inputs=x, filters=6, kernel_size=(5,5), strides=(1,1), padding='valid', name='conv1')
conv1 = tf.nn.relu(conv1)
conv1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=(2,2), strides=(2,2), padding='valid')

conv2 = tf.layers.conv2d(inputs=conv1, filters=16, kernel_size=(5,5), strides=(1,1), padding='valid', name='conv2')
conv2 = tf.nn.relu(conv2)
conv2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=(2,2), strides=(2,2), padding='valid')

flatten = tf.layers.flatten(conv2)

fc1 = tf.layers.dense(flatten, units=120, use_bias=True, activation=tf.nn.relu, name='fc1')
fc2 = tf.layers.dense(fc1, units=84, use_bias=True, activation=tf.nn.relu, name='fc2')
embedding_layer = tf.layers.dense(fc2, units=params['embedding'], use_bias=True, activation=None)

clf_layer = tf.layers.dense(embedding_layer, units=params['n_classes'], activation=None, use_bias=False, name='clf_layer')

with tf.variable_scope('clf_layer', reuse=True):
    weights = tf.get_variable('kernel', trainable=True, regularizer=slim.l2_regularizer(1e-5))

logits = calculate_arcface_logits(embedding_layer=embedding_layer, weights=weights, y=y_ohe,
                                     class_num=params['n_classes'], s=params['logits_scale'], m=params['logits_margin'])
softmax_layer = tf.nn.softmax(logits)

W0830 18:04:16.949633 140602272642816 deprecation.py:323] From <ipython-input-4-3b35288236eb>:8: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0830 18:04:16.961263 140602272642816 deprecation.py:506] From /home/kim1/anaconda3/envs/rok/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0830 18:04:17.189656 140602272642816 deprecation.py:323] From <ipython-input-4-3b35288236eb>:10: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0830 18:04:17.332782 140602272642816 dep

In [5]:
print('conv1:{}'.format(conv1.shape))
print('conv2:{}'.format(conv2.shape))
print('fc1:{}'.format(fc1.shape))
print('fc2:{}'.format(fc2.shape))
print('embedding_layer:{}'.format(embedding_layer.shape))
print('clf_layer:{}'.format(clf_layer.shape))

conv1:(?, 14, 14, 6)
conv2:(?, 5, 5, 16)
fc1:(?, 120)
fc2:(?, 84)
embedding_layer:(?, 16)
clf_layer:(?, 10)


In [6]:
eta = 1e-4
epsilon = 1e-5
loss = tf.reduce_sum(tf.maximum(tf.multiply(-tf.log(softmax_layer + epsilon), y_ohe), 0))
train_model = tf.train.GradientDescentOptimizer(learning_rate=eta).minimize(loss)

W0830 18:04:18.046915 140602272642816 deprecation.py:323] From /home/kim1/anaconda3/envs/rok/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<br>
Training

In [7]:
def cosine_distance(centroid, sample):
    inner_product = np.dot(sample, centroid)
    centroid_dist = np.sqrt(np.sum(np.square(centroid)))
    try:
        sample_dist = np.sqrt(np.sum(np.square(sample), axis=1))
    except:
        sample_dist = np.sqrt(np.sum(np.square(sample)))
    return 1 - (inner_product / (centroid_dist * sample_dist))

In [8]:
loss_path_train = []
loss_path_test = []

embedding_train = []
embedding_test = []

accuracy_train = []
accuracy_test = []

weight_train = []

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
iter_cnt = 50
np.random.seed(params['seed'])
sample_size = 10000 # use 10000 samples only for monitoring

# initial embedding train/test 
embedding_train.append(sess.run(embedding_layer, feed_dict={x:x_train[:sample_size]}))
embedding_test.append(sess.run(embedding_layer, feed_dict={x:x_test[:sample_size]})) 

# weight train/test
weight_train.append(sess.run(weights))

for epoch in range(iter_cnt):
    start_time = time.time()
    cursor = 0
    step = 1
    
    # random shuffle
    train_idx = np.arange(len(y_train))
    np.random.shuffle(train_idx)
    shuffled_x_train = x_train[train_idx]
    shuffled_y_train_ohe = y_train_ohe[train_idx]

    while cursor < len(y_train): 
        batch_x_train = shuffled_x_train[cursor:cursor+params['batch_size']]
        batch_y_train_ohe = shuffled_y_train_ohe[cursor:cursor+params['batch_size']] 
        sess.run(train_model, feed_dict={x:batch_x_train, y_ohe:batch_y_train_ohe})       
        step += 1
        cursor += params['batch_size']
    
    # embedding train/test
    embedding_train.append(sess.run(embedding_layer, feed_dict={x:x_train[:sample_size]}))
    embedding_test.append(sess.run(embedding_layer, feed_dict={x:x_test[:sample_size]}))
    
    # loss train/test
    loss_path_train.append(sess.run(loss, feed_dict={x:x_train[:sample_size], y_ohe:y_train_ohe[:sample_size]}))
    loss_path_test.append(sess.run(loss, feed_dict={x:x_test[:sample_size], y_ohe:y_test_ohe[:sample_size]}))
    
    # weight
    weight_train.append(sess.run(weights))
    
    # accuracy train/test
    train_distance_list = []
    test_distance_list = []
    for c in weight_train[-1].T:
        train_distance_list.append(cosine_distance(centroid=c, sample=embedding_train[-1][:sample_size]))
        test_distance_list.append(cosine_distance(centroid=c, sample=embedding_test[-1][:sample_size]))
    pred_train = pd.DataFrame(np.array(train_distance_list).T).idxmin(axis=1).values
    pred_test = pd.DataFrame(np.array(test_distance_list).T).idxmin(axis=1).values
    accuracy_train.append(np.sum((pred_train == y_train[:sample_size]).astype('int')) / len(y_train[:sample_size]))
    accuracy_test.append(np.sum((pred_test == y_test[:sample_size]).astype('int')) / len(y_test[:sample_size]))
        
    end_time = time.time()
    
    print('epoch:{}    {:.2f}sec \n\
           train(loss:{:.4f}, accuracy{:.4f}) \n\
           test (loss:{:.4f}, accuracy{:.4f})'.format(
           epoch+1, end_time - start_time,
           loss_path_train[-1], accuracy_train[-1],
           loss_path_test[-1], accuracy_test[-1]))
    print('')
sess.close()

epoch:1    13.39sec 
           train(loss:28703.4727, accuracy0.2640) 
           test (loss:28681.1152, accuracy0.2670)

epoch:2    14.74sec 
           train(loss:24264.6719, accuracy0.4028) 
           test (loss:24416.9707, accuracy0.4026)

epoch:3    13.20sec 
           train(loss:22335.3672, accuracy0.4520) 
           test (loss:22646.1953, accuracy0.4476)

epoch:4    14.85sec 
           train(loss:21101.5820, accuracy0.4917) 
           test (loss:21523.3320, accuracy0.4772)

epoch:5    13.21sec 
           train(loss:20124.7383, accuracy0.5194) 
           test (loss:20676.2070, accuracy0.4996)

epoch:6    14.77sec 
           train(loss:19459.2129, accuracy0.5341) 
           test (loss:20076.4668, accuracy0.5111)

epoch:7    13.21sec 
           train(loss:18921.8164, accuracy0.5472) 
           test (loss:19629.5781, accuracy0.5236)

epoch:8    14.76sec 
           train(loss:18893.2812, accuracy0.5463) 
           test (loss:19648.8867, accuracy0.5200)

epoch:9    13.29